# Volume 3: MongoDB.
    Ben Christensen
    Math 403
    December 18, 2018

In [1]:
import re
import json
from pymongo import MongoClient
from datetime import datetime
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [2]:
#Load the json file of Trump tweets
tweets = list()
with open("trump.json", 'r') as infile:
    for line in infile:
        tweets.append(json.loads(line))    

In [3]:
#Initialize the Mongo database and make sure its collection is empty
client = MongoClient()
db = client.db1
col = db.collection1
col.delete_many(dict())

In [4]:
#Write trump tweets to the collection
for tweet in tweets:
    col.insert_one(tweet)

In [5]:
col.count()

95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

### (i)

In [6]:
#Find every tweet that mentions Russia
results = col.count_documents({"text":{"$regex":re.compile("russia", re.IGNORECASE)}})

In [7]:
print("The number of tweets including the word 'Russia':")
print(results)

The number of tweets including the word 'Russia':
5841


### (ii)

In [8]:
#Find the number of tweets coming from the following time zones
US_time_zones = ["Central Time (US & Canada)", "Pacific Time (US & Canada)",
                 "Eastern Time (US & Canada)", "Mountain Time (US & Canada)"]
results = col.count_documents({'user.time_zone': {'$in':US_time_zones}})

In [9]:
print("The number of tweets coming from the main continental US time zones:")
print(results)

The number of tweets coming from the main continental US time zones:
33939


In [10]:
#Find the number of tweets written in each language
languages = col.distinct('lang')
lang_counts = dict()
for lang in languages:
    lang_counts[lang] = col.count_documents({'lang':lang})
    print(lang,': ', lang_counts[lang], sep='')


en: 84919
fr: 1235
es: 3027
tr: 588
nl: 212
pt: 632
und: 1653
it: 423
de: 635
da: 35
zh: 1
ru: 29
pl: 108
no: 21
eu: 6
tl: 48
in: 121
cs: 13
ht: 21
ro: 21
sl: 3
sv: 55
fi: 20
ar: 30
et: 23
lt: 8
hu: 6
el: 10
ja: 17
lv: 2
vi: 7
fa: 5
ko: 2
ur: 1
cy: 2
th: 3
is: 1
iw: 1
uk: 3


## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

### (i)

In [11]:
#Find 5 most popular tweeters mentioning trump
print("Top 5 Most Popular Tweeters with their number of followers")
results = col.find({'user.followers_count':{'$exists':True}}).sort('user.followers_count', -1)
tweet = results[0]
print(tweet['user']['name'], ': ', tweet['user']['followers_count'], sep='')
tweet = results[1]
print(tweet['user']['name'], ': ', tweet['user']['followers_count'], sep='')
tweet = results[2]
print(tweet['user']['name'], ': ', tweet['user']['followers_count'], sep='')
tweet = results[3]
print(tweet['user']['name'], ': ', tweet['user']['followers_count'], sep='')
#4 and 5 are also Wall Street Journal
tweet = results[6]
print(tweet['user']['name'], ': ', tweet['user']['followers_count'], sep='')

Top 5 Most Popular Tweeters with their number of followers
CNN: 35481010
Reuters Top News: 17821353
Fox News: 14663497
Wall Street Journal: 14023091
TIME: 14017299


### (ii)

In [12]:
#Sort the tweets to find the tweet with the earliest 5th hashtag
results = col.find({'entities.hashtags.4':{'$exists':True}}).sort('entities.hashtags.4.indices.0')

In [13]:
print("The earliest spot the 5th hashtag appears:")
print(results[0]['entities']['hashtags'][4]['indices'][0])

The earliest spot the 5th hashtag appears:
23


### (iii)

In [14]:
#find the tweet that came from the noternmost location
results = col.find({'coordinates.coordinates.0':{'$ne':None}}).sort('coordinates.coordinates.0', -1)

In [15]:
print("Coordinates for the tweet that came from the northernmost location:")
print(results[0]['coordinates']['coordinates'])

Coordinates for the tweet that came from the northernmost location:
[28.99237342, 41.06803023]


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.
- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [16]:
#Remove the 'retweeted_status' field from every tweet
#print(col.count_documents({'retweeted_status':{'$exists':True}}))
col.update_many({'retweeted_status':{'$exists':True}}, {'$unset': {'retweeted_status':''}})
#print(col.count_documents({'retweeted_status':{'$exists':True}}))

In [17]:
#Add an attribute called 'popular' that is True if user has at least
#    1000 followers.
#print(col.count_documents({'user.followers_count':{'$gte':1000}}))
col.update_many({'user.followers_count':{'$gte':1000}}, {'$set':{'popular':True}})
print("The Number of popular tweets (>= 1000 followers)")
print(col.count_documents({'popular':True}))

The Number of popular tweets (>= 1000 followers)
32489
